In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import random
from gurobipy import *
from scipy.spatial.distance import cdist, euclidean
from scipy.stats import truncnorm
from scipy.sparse import csr_matrix
from time import time
import networkx as nx

import pickle


#path = "C:/Users/a.rojasa55/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"
#path = "C:/Users/ari_r/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"
path = '/Users/juanbeta/My Drive/Research/MOPTA/MOPTA-23/Data/'

vehicles = pd.read_csv(path+'MOPTA2023_car_locations.csv', sep = ',', header = None)
vehicles = vehicles.append([vehicles]*9,ignore_index=True)

stations = pd.read_csv(path+"fuel_stations.csv")

northern = (-79.761960, 42.269385)
southern = (-76.9909,39.7198)
western = (-80.519400, 40.639400)
eastern = (-74.689603, 41.363559)

stations_loc = stations[["Longitude","Latitude"]]
stations_loc["Latitude"] = (stations["Latitude"]-southern[1])*69*165/178
stations_loc["Longitude"] = (stations["Longitude"]-western[0])*53

stations = stations_loc[(stations_loc["Longitude"] <= 290) & (stations_loc["Latitude"] <= 150)]
stations.rename(columns={"Longitude": 0, "Latitude":1}, inplace=True)

/var/folders/m6/89fwm0dn5mjbdz4tkj6586bm0000gn/T/ipykernel_3173/1303529806.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vehicles = vehicles.append([vehicles]*9,ignore_index=True)
/var/folders/m6/89fwm0dn5mjbdz4tkj6586bm0000gn/T/ipykernel_3173/1303529806.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations_loc["Latitude"] = (stations["Latitude"]-southern[1])*69*165/178
/var/folders/m6/89fwm0dn5mjbdz4tkj6586bm0000gn/T/ipykernel_3173/1303529806.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [2]:
loc, scale, min_v, max_v= 100, 50, 20, 250
a, b = (min_v - loc) / scale, (max_v - loc) / scale

def generate_distances():
    distances = dict()
    for i in vehicles.index:
        for st in stations.index:    
            distances[st,i] = euclidean([vehicles[0][i], vehicles[1][i]],[stations[0][st], stations[1][st]])
    return distances

def generate_feasible_instance(distances):
    flag = True
    ii = -1
    while flag:
        ii += 1
        flag = False
        total_v = 0
        ranges, realized = list(), list()

        for i in vehicles.index:
            feas = False
            rr = truncnorm.rvs(a = a, b = b, loc = loc, scale = scale, size = 1)
            ranges.append(rr[0])
            prob = np.exp(-0.012**2*(rr-20)**2)
            realization = random.choice([True,False], p = [prob[0], 1-prob[0]])
            realized.append(realization)
            if realization == True:
                total_v += 1
                for st in stations.index:    
                    if distances[st,i] <= rr:
                        feas = True

                if not feas:
                    flag = True
                    print(f'\tTry n. {ii} failed at vehicle {i}')
                    break
    
    print(f'\tTry n. {ii} succesfull - {total_v} realized EV')

    return ranges, realized, total_v

In [3]:
random.seed(0)

d_matrix = generate_distances()
total_ev = dict()

realizations = list()
for instance in range(25):
    print(f'\nGENERATING INSTANCE N.o {instance}')
    ranges, realized, total_v = generate_feasible_instance(d_matrix)

    realizations.append((ranges,realized))

    vehicles[f'ranges {instance}'] = ranges
    vehicles[f'realized {instance}'] = realized

    total_ev[instance] = total_v

data0 = [total_ev, realizations]
data = [total_ev, vehicles]

a_file = open(path + f'feasible instances0', "wb")
pickle.dump(data0, a_file)
a_file.close()


a_file = open(path + f'feasible instances', "wb")
pickle.dump(data, a_file)
a_file.close()


GENERATING INSTANCE N.o 0
	Try n. 0 failed at vehicle 4297
	Try n. 1 failed at vehicle 4297
	Try n. 2 failed at vehicle 8600
	Try n. 3 succesfull - 4590 realized EV

GENERATING INSTANCE N.o 1
	Try n. 0 failed at vehicle 5368
	Try n. 1 succesfull - 4527 realized EV

GENERATING INSTANCE N.o 2
	Try n. 0 failed at vehicle 2126
	Try n. 1 failed at vehicle 5363
	Try n. 2 failed at vehicle 8604
	Try n. 3 succesfull - 4471 realized EV

GENERATING INSTANCE N.o 3
	Try n. 0 succesfull - 4551 realized EV

GENERATING INSTANCE N.o 4
	Try n. 0 succesfull - 4474 realized EV

GENERATING INSTANCE N.o 5
	Try n. 0 failed at vehicle 3218
	Try n. 1 succesfull - 4498 realized EV

GENERATING INSTANCE N.o 6
	Try n. 0 failed at vehicle 1052
	Try n. 1 failed at vehicle 10763
	Try n. 2 succesfull - 4628 realized EV

GENERATING INSTANCE N.o 7
	Try n. 0 succesfull - 4565 realized EV

GENERATING INSTANCE N.o 8
	Try n. 0 failed at vehicle 1060
	Try n. 1 succesfull - 4633 realized EV

GENERATING INSTANCE N.o 9
	Try n

In [4]:
file = open(path + f'feasible instances', 'rb')
data = pickle.load(file)
file.close()